# Overview

This notebook aggregates country-level carbon storage data into IAMC region definitions

In [ ]:
import pandas as pd
from pandas_indexing import ismatch

In [57]:
def cols_to_dict(df, col1, col2):
    keys = df[col1].unique()
    df = df.set_index(col1)
    return {key: df[col2][key].to_list() for key in keys}

mdf = pd.read_csv('../raw_data/iso3c_region_mapping_20240319_highlighted.csv')
r5_mapping = cols_to_dict(mdf.dropna(subset='r5_iamc'), 'r5_iamc', 'iso3c')
r10_mapping = cols_to_dict(mdf.dropna(subset='r10_iamc'), 'r10_iamc', 'iso3c')
w_mapping = {'World': mdf.iso3c.unique()}

mdf.head()

,iso3c,m49code,r5_iamc,r6_ipcc,r10_unif,r10_iamc,r10_ipcc,r10_message,r11_message,r12_message,...,m49lo_ipcc,m49dev_ipcc,m49sids_ipcc,m49lldc_ipcc,m49ldc_ipcc,annexi_ipcc,wmo_ipcc,eu_ipcc,oecd_ipcc,wb_ipcc
0,AGO,24.0,R5MAF,Africa,R10AFRMEA,R10AFRICA,Africa,Sub-saharan Africa,Sub-saharan Africa,R12AFR,...,Middle Africa,Developing,NaN,NaN,LDC,Non-Annex I,I,NaN,NaN,Lower-middle
1,ARE,784.0,R5MAF,Middle East,R10AFRMEA,R10MIDDLE_EAST,Middle East,Middle East and North Africa,Middle East and North Africa,R12MEA,...,Western Asia,Developing,NaN,NaN,NaN,Non-Annex I,II,NaN,NaN,High
2,ATF,260.0,NaN,Africa,R10AFRMEA,NaN,Africa,NaN,NaN,NaN,...,Eastern Africa,Developing,NaN,NaN,NaN,Territory,NaN,NaN,NaN,NaN
3,BDI,108.0,R5MAF,Africa,R10AFRMEA,R10AFRICA,Africa,Sub-saharan Africa,Sub-saharan Africa,R12AFR,...,Eastern Africa,Developing,NaN,LLDC,LDC,Non-Annex I,I,NaN,NaN,Low
4,BEN,204.0,R5MAF,Africa,R10AFRMEA,R10AFRICA,Africa,Sub-saharan Africa,Sub-saharan Africa,R12AFR,...,Western Africa,Developing,NaN,NaN,LDC,Non-Annex I,I,NaN,NaN,Lower-middle


In [56]:
df = pd.read_csv('../raw_data/D2_compiled.csv', index_col='ISO')
df.head()

,Absolute storage loss (net vs gross),Percentage lost (net vs gross),Total storage loss - Geogaphical exclusion zones,Total storage loss - policy exclusion zones,Net total storage,Gross total storage
ISO,,,,,,
ZWE,0.00,0.000000,0.00,0.00,0.10,0.10
ZNC,0.17,100.000000,0.17,0.00,0.00,0.17
ZMB,1.07,27.506427,0.00,1.07,2.82,3.89
ZAF,14.29,52.575423,10.61,3.68,12.89,27.18
YEM,2.20,39.783002,1.71,0.49,3.33,5.53


In [58]:
def aggregate(df, mapping):
    df = df.pix.aggregate(ISO=mapping, mode='return')
    df['Percentage lost (net vs gross)'] = df['Net total storage'] / df['Gross total storage']
    return df.rename_axis(index={'ISO': 'Region'})

data = pd.concat([
    aggregate(df, r5_mapping),
    aggregate(df, r10_mapping),
    aggregate(df, w_mapping),
])

data

,Absolute storage loss (net vs gross),Percentage lost (net vs gross),Total storage loss - Geogaphical exclusion zones,Total storage loss - policy exclusion zones,Net total storage,Gross total storage
Region,,,,,,
R5ASIA,270.76,0.401887,223.95,46.81,181.93,452.69
R5LAM,226.61,0.479679,158.15,68.46,208.91,435.52
R5MAF,316.46,0.553552,167.55,148.91,392.38,708.84
R5OECD90+EU,575.53,0.432226,370.87,204.66,438.13,1013.66
R5REF,350.09,0.479141,84.61,265.48,322.05,672.14
R10AFRICA,265.06,0.547223,143.05,122.01,320.35,585.41
R10CHINA+,60.84,0.605089,45.36,15.48,93.22,154.06
R10EUROPE,248.48,0.140505,154.13,94.35,40.62,289.10
R10INDIA+,102.65,0.166667,90.26,12.39,20.53,123.18


In [59]:
data.to_csv('../processed_data/101_D2_compiled_r5_r10.csv', index=True)